In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib

pd.set_option("display.max_rows", None, "display.max_columns", None)

def assignTeamToPlayerId(df):
    team_by_id = {}
    for i in range(len(df)):
        
        if df['player1Id'].iloc[i] not in team_by_id:
            tn = df['player1Team'].iloc[i]
            team_by_id[df['player1Id'].iloc[i]] = tn
        if df['player2Id'].iloc[i] not in team_by_id:
            tn = df['player2Team'].iloc[i]
            team_by_id[df['player2Id'].iloc[i]] = tn
        if df['player3Id'].iloc[i] not in team_by_id:
            tn = df['player3Team'].iloc[i]
            team_by_id[df['player3Id'].iloc[i]] = tn
        if df['player4Id'].iloc[i] not in team_by_id:
            tn = df['player4Team'].iloc[i]
            team_by_id[df['player4Id'].iloc[i]] = tn
        if df['player5Id'].iloc[i] not in team_by_id:
            tn = df['player5Team'].iloc[i]
            team_by_id[df['player5Id'].iloc[i]] = tn
        if df['player6Id'].iloc[i] not in team_by_id:
            tn = df['player6Team'].iloc[i]
            team_by_id[df['player6Id'].iloc[i]] = tn
        if df['player7Id'].iloc[i] not in team_by_id:
            tn = df['player7Team'].iloc[i]
            team_by_id[df['player7Id'].iloc[i]] = tn
        if df['player8Id'].iloc[i] not in team_by_id:
            tn = df['player8Team'].iloc[i]
            team_by_id[df['player8Id'].iloc[i]] = tn
        if df['player9Id'].iloc[i] not in team_by_id:
            tn = df['player9Team'].iloc[i]
            team_by_id[df['player9Id'].iloc[i]] = tn
        if df['player10Id'].iloc[i] not in team_by_id:
            tn = df['player10Team'].iloc[i]
            team_by_id[df['player10Id'].iloc[i]] = tn
            
    return team_by_id

def DictListUpdate( lis1, lis2):
    for aLis1 in lis1:
        if aLis1 not in lis2:
            lis2.append(aLis1)
    return lis2

conn = sqlite3.connect('NAStage1Challengers.db')
cursor = conn.cursor()

In [2]:
#Load all needed tables
series = pd.read_sql('SELECT * FROM series',conn)
players = pd.read_sql('SELECT * FROM players',conn)
pStats = pd.read_sql('SELECT * FROM pStats', conn)
xvy = pd.read_sql('SELECT * FROM xvy', conn)
kills = pd.read_sql('SELECT * FROM kills', conn)
matches = pd.read_sql('SELECT * FROM matches', conn)
rounds = pd.read_sql('SELECT * FROM rounds', conn)
events = pd.read_sql('SELECT * FROM details', conn)

In [3]:
#load team names + ids and make a dictionary by IDs
series = series[['seriesId','team1Id','team1Name','team2Id','team2Name']]

teamDict1 = dict(zip(series.team1Id,series.team1Name))
teamDict2 = dict(zip( series.team2Id,series.team2Name))
teamDict = teamDict1 | teamDict2
len(teamDict)

12

In [4]:
#load player names + ids and assign a team to the id in a dictionary
playersTeamsDict = assignTeamToPlayerId(players)

pStats['playerTeam'] = pStats['playerId'].map(playersTeamsDict)
len(playersTeamsDict)

62

In [5]:
#grab match id and map for final stats
matches = matches[['matchId','map']]

matches.head()

,matchId,map
0,58155,Ascent
1,58156,Breeze
2,58157,Haven
3,58158,Split
4,58159,Ascent


In [6]:
#merge player stats + players
pStatsFinal = pStats.merge(players,on='matchId', how='left')
pStatsFinal.drop_duplicates(inplace=True)
len(pStatsFinal)

23870

In [7]:
#add map
pStatsFinal = pStatsFinal.merge(matches, on='matchId', how='left')
pStatsFinal.drop_duplicates(inplace=True)
pStatsFinal.drop(['index_x','index_y'], axis=1, inplace=True)
len(pStatsFinal)

23870

In [8]:
#Append player name + player agent to end of final stats
pStatsFinal['playerStats'] = ""
pStatsFinal['agentStats'] = ""


for i in range(len(pStatsFinal)):
    pId = pStatsFinal.iloc[i,3]
    for ii in range(24,64,4):
        if pId == pStatsFinal.iloc[i,ii]:
            pStatsFinal.iloc[i,65] = pStatsFinal.iloc[i,ii+1]
            pStatsFinal.iloc[i,66] = pStatsFinal.iloc[i,ii+2]
            pStatsFinal.iloc[i,23] = pStatsFinal.iloc[i,ii+3]
            break
            


In [9]:
pStatsFinal.head()



,matchId,roundId,roundNumber,playerId,teamNumber,side,acs,kills,firstKills,deaths,firstDeaths,assists,damage,headshots,bodyshots,legshots,plants,defusals,clutches,clutchOpponents,clutchOpportunities,impact,kastRounds,playerTeam,player1Id,player1Name,player1AgentName,player1Team,player2Id,player2Name,player2AgentName,player2Team,player3Id,player3Name,player3AgentName,player3Team,player4Id,player4Name,player4AgentName,player4Team,player5Id,player5Name,player5AgentName,player5Team,player6Id,player6Name,player6AgentName,player6Team,player7Id,player7Name,player7AgentName,player7Team,player8Id,player8Name,player8AgentName,player8Team,player9Id,player9Name,player9AgentName,player9Team,player10Id,player10Name,player10AgentName,player10Team,map,playerStats,agentStats
0,58155,906584,1,835,2,atk,272,1,1,1,0,0,126,1,1,1,1,0,0,0,0,0.11063158287637609,1,Knights,1961,xeta,Sova,Cloud9,4639,vanity,Omen,Cloud9,13591,Xeppaa,Kayo,Cloud9,3599,leaf,Jett,Cloud9,2213,mitch,Killjoy,Cloud9,835,XXiF,Sage,Knights,2810,Genghsta,Sova,Knights,3340,ban,Jett,Knights,3831,skuba,Astra,Knights,3894,Frosty,Killjoy,Knights,Ascent,XXiF,Sage
1,58155,906584,1,1961,1,def,734,3,0,1,0,0,270,2,2,0,0,0,0,3,1,0.06115419295030323,1,Cloud9,1961,xeta,Sova,Cloud9,4639,vanity,Omen,Cloud9,13591,Xeppaa,Kayo,Cloud9,3599,leaf,Jett,Cloud9,2213,mitch,Killjoy,Cloud9,835,XXiF,Sage,Knights,2810,Genghsta,Sova,Knights,3340,ban,Jett,Knights,3831,skuba,Astra,Knights,3894,Frosty,Killjoy,Knights,Ascent,xeta,Sova
2,58155,906584,1,2213,1,def,20,0,0,1,0,0,20,0,2,0,0,0,0,0,0,-0.08521074372468429,0,Cloud9,1961,xeta,Sova,Cloud9,4639,vanity,Omen,Cloud9,13591,Xeppaa,Kayo,Cloud9,3599,leaf,Jett,Cloud9,2213,mitch,Killjoy,Cloud9,835,XXiF,Sage,Knights,2810,Genghsta,Sova,Knights,3340,ban,Jett,Knights,3831,skuba,Astra,Knights,3894,Frosty,Killjoy,Knights,Ascent,mitch,Killjoy
3,58155,906584,1,2810,2,atk,183,0,0,0,0,2,183,1,1,0,0,0,0,0,0,0,1,Knights,1961,xeta,Sova,Cloud9,4639,vanity,Omen,Cloud9,13591,Xeppaa,Kayo,Cloud9,3599,leaf,Jett,Cloud9,2213,mitch,Killjoy,Cloud9,835,XXiF,Sage,Knights,2810,Genghsta,Sova,Knights,3340,ban,Jett,Knights,3831,skuba,Astra,Knights,3894,Frosty,Killjoy,Knights,Ascent,Genghsta,Sova
4,58155,906584,1,3340,2,atk,792,4,0,0,0,0,397,4,4,0,0,0,0,0,0,0.42696423585487474,1,Knights,1961,xeta,Sova,Cloud9,4639,vanity,Omen,Cloud9,13591,Xeppaa,Kayo,Cloud9,3599,leaf,Jett,Cloud9,2213,mitch,Killjoy,Cloud9,835,XXiF,Sage,Knights,2810,Genghsta,Sova,Knights,3340,ban,Jett,Knights,3831,skuba,Astra,Knights,3894,Frosty,Killjoy,Knights,Ascent,ban,Jett


In [10]:
pStatsFinal.drop_duplicates(inplace=True)
pStatsFinal.drop(pStatsFinal.columns[24:64], axis=1, inplace=True)
len(pStatsFinal)

23870

# GET FK / FD TIMES

In [11]:

kills.rename(columns={'killerId':'playerId'}, inplace=True)
kills.head()

,index,matchId,id,roundId,playerId,victimId,roundTimeMillis,gameTimeMillis,victimLocationX,victimLocationY,damageType,abilityType,weaponId,secondaryFireMode,first,tradedByKillId,tradedForKillId,weapon,weaponCategory,killerTeamNumber,victimTeamNumber,side
0,0,58155,7632163,906584,3340,13591,29960,179358,5372,-7739,weapon,None,10.0,0,0,NaN,NaN,Frenzy,sidearm,2,1,atk
1,1,58155,7632164,906584,3340,2213,33947,183345,6056,-8328,weapon,None,10.0,0,0,NaN,NaN,Frenzy,sidearm,2,1,atk
2,2,58155,7632165,906584,3340,4639,44771,194169,6682,-7411,weapon,None,10.0,0,0,NaN,NaN,Frenzy,sidearm,2,1,atk
3,3,58155,7632166,906584,3340,1961,46691,196089,5558,-7509,weapon,None,10.0,0,0,NaN,7632170.0,Frenzy,sidearm,2,1,atk
4,4,58155,7632167,906584,835,3599,28277,177675,5246,-7606,weapon,None,11.0,0,1,NaN,NaN,Classic,sidearm,2,1,atk


In [12]:
len(kills)

16707

In [13]:
kills = kills[kills['first'] == 1]
kills = kills[['roundId','playerId','roundTimeMillis','victimId']]
kills.head()

,roundId,playerId,roundTimeMillis,victimId
4,906584,835,28277,3599
8,906585,3340,26329,1961
15,906586,3599,14954,3340
21,906587,2213,26259,2810
26,906588,2213,61709,3340


In [14]:
test = pStatsFinal.merge(kills, on=['playerId','roundId'], how='left')

test.head()

,matchId,roundId,roundNumber,playerId,teamNumber,side,acs,kills,firstKills,deaths,firstDeaths,assists,damage,headshots,bodyshots,legshots,plants,defusals,clutches,clutchOpponents,clutchOpportunities,impact,kastRounds,playerTeam,map,playerStats,agentStats,roundTimeMillis,victimId
0,58155,906584,1,835,2,atk,272,1,1,1,0,0,126,1,1,1,1,0,0,0,0,0.11063158287637609,1,Knights,Ascent,XXiF,Sage,28277.0,3599.0
1,58155,906584,1,1961,1,def,734,3,0,1,0,0,270,2,2,0,0,0,0,3,1,0.06115419295030323,1,Cloud9,Ascent,xeta,Sova,NaN,NaN
2,58155,906584,1,2213,1,def,20,0,0,1,0,0,20,0,2,0,0,0,0,0,0,-0.08521074372468429,0,Cloud9,Ascent,mitch,Killjoy,NaN,NaN
3,58155,906584,1,2810,2,atk,183,0,0,0,0,2,183,1,1,0,0,0,0,0,0,0,1,Knights,Ascent,Genghsta,Sova,NaN,NaN
4,58155,906584,1,3340,2,atk,792,4,0,0,0,0,397,4,4,0,0,0,0,0,0,0.42696423585487474,1,Knights,Ascent,ban,Jett,NaN,NaN


In [15]:
kills.drop(columns=['playerId'],inplace=True,axis=1)

In [16]:
kills.rename(columns={'victimId':'playerId'}, inplace=True)
kills.head()


,roundId,roundTimeMillis,playerId
4,906584,28277,3599
8,906585,26329,1961
15,906586,14954,3340
21,906587,26259,2810
26,906588,61709,3340


In [17]:
test = test.merge(kills, on=['playerId','roundId'], how='left')
test.roundTimeMillis_x.fillna(test.roundTimeMillis_y, inplace=True)
test.drop(columns=['victimId','roundTimeMillis_y'], inplace=True)
test.rename(columns={'roundTimeMillis_x':'firstDuelTime'},inplace=True)
test.drop_duplicates(inplace=True)
test.head()

,matchId,roundId,roundNumber,playerId,teamNumber,side,acs,kills,firstKills,deaths,firstDeaths,assists,damage,headshots,bodyshots,legshots,plants,defusals,clutches,clutchOpponents,clutchOpportunities,impact,kastRounds,playerTeam,map,playerStats,agentStats,firstDuelTime
0,58155,906584,1,835,2,atk,272,1,1,1,0,0,126,1,1,1,1,0,0,0,0,0.11063158287637609,1,Knights,Ascent,XXiF,Sage,28277.0
1,58155,906584,1,1961,1,def,734,3,0,1,0,0,270,2,2,0,0,0,0,3,1,0.06115419295030323,1,Cloud9,Ascent,xeta,Sova,NaN
2,58155,906584,1,2213,1,def,20,0,0,1,0,0,20,0,2,0,0,0,0,0,0,-0.08521074372468429,0,Cloud9,Ascent,mitch,Killjoy,NaN
3,58155,906584,1,2810,2,atk,183,0,0,0,0,2,183,1,1,0,0,0,0,0,0,0,1,Knights,Ascent,Genghsta,Sova,NaN
4,58155,906584,1,3340,2,atk,792,4,0,0,0,0,397,4,4,0,0,0,0,0,0,0.42696423585487474,1,Knights,Ascent,ban,Jett,NaN


In [18]:
print(len(test), len(pStatsFinal))

23870 23870


In [19]:
##############################
#ROUND CLEANING + MERGE BEGIN
#############################

#Grab only necesary columns then turn NaNs into 0s and turn ints into readable strings
#0 = pistol | 1 = save | 2 = light buy | 3 = mid buy | 4 = Full buy
rounds = rounds[['id','winningTeamNumber','team1LoadoutTier','team2LoadoutTier']]

rounds.fillna(0, inplace=True)

rounds['team1LoadoutTier'] = rounds['team1LoadoutTier']
rounds['team2LoadoutTier'] = rounds['team2LoadoutTier']

#rounds.replace({'0.0':'Pistol','1.0':'Save','2.0':'Light','3.0':'Half','4.0':'Full'},inplace=True)

rounds.rename(columns= {'id':'roundId'},inplace=True)

rounds.head()
len(rounds)

2387

In [20]:
print(len(test), len(pStatsFinal))

23870 23870


In [21]:
#Merge round info to pStats

pStatsFinal = test.merge(rounds,on='roundId',how='left')
pStatsFinal.drop_duplicates(inplace=True)
pStatsFinal.reset_index(inplace=True)
#pStatsFinal.head()
len(pStatsFinal)

23870

In [22]:

pStatsFinal.head(20)

,index,matchId,roundId,roundNumber,playerId,teamNumber,side,acs,kills,firstKills,deaths,firstDeaths,assists,damage,headshots,bodyshots,legshots,plants,defusals,clutches,clutchOpponents,clutchOpportunities,impact,kastRounds,playerTeam,map,playerStats,agentStats,firstDuelTime,winningTeamNumber,team1LoadoutTier,team2LoadoutTier
0,0,58155,906584,1,835,2,atk,272,1,1,1,0,0,126,1,1,1,1,0,0,0,0,0.11063158287637609,1,Knights,Ascent,XXiF,Sage,28277.0,2,0.0,0.0
1,1,58155,906584,1,1961,1,def,734,3,0,1,0,0,270,2,2,0,0,0,0,3,1,0.06115419295030323,1,Cloud9,Ascent,xeta,Sova,NaN,2,0.0,0.0
2,2,58155,906584,1,2213,1,def,20,0,0,1,0,0,20,0,2,0,0,0,0,0,0,-0.08521074372468429,0,Cloud9,Ascent,mitch,Killjoy,NaN,2,0.0,0.0
3,3,58155,906584,1,2810,2,atk,183,0,0,0,0,2,183,1,1,0,0,0,0,0,0,0,1,Knights,Ascent,Genghsta,Sova,NaN,2,0.0,0.0
4,4,58155,906584,1,3340,2,atk,792,4,0,0,0,0,397,4,4,0,0,0,0,0,0,0.42696423585487474,1,Knights,Ascent,ban,Jett,NaN,2,0.0,0.0
5,5,58155,906584,1,3599,1,def,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,-0.18277501265111362,0,Cloud9,Ascent,leaf,Jett,28277.0,2,0.0,0.0
6,6,58155,906584,1,3831,2,atk,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-0.11331540950854158,1,Knights,Ascent,skuba,Astra,NaN,2,0.0,0.0
7,7,58155,906584,1,3894,2,atk,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-0.03620138165895892,0,Knights,Ascent,Frosty,Killjoy,NaN,2,0.0,0.0
8,8,58155,906584,1,4639,1,def,74,0,0,1,0,1,74,0,2,1,0,0,0,0,0,-0.09899708791569006,1,Cloud9,Ascent,vanity,Omen,NaN,2,0.0,0.0
9,9,58155,906584,1,13591,1,def,26,0,0,1,0,0,26,0,1,0,0,0,0,0,0,-0.08225037622256559,0,Cloud9,Ascent,Xeppaa,Kayo,NaN,2,0.0,0.0


In [23]:
pStatsFinal['teamLoadout'] = np.where(pStatsFinal['teamNumber'] == 1, pStatsFinal['team1LoadoutTier'], pStatsFinal['team2LoadoutTier'])
pStatsFinal['enemyLoadout'] = np.where(pStatsFinal['teamNumber'] != 1, pStatsFinal['team1LoadoutTier'], pStatsFinal['team2LoadoutTier'])
pStatsFinal.drop(columns=['team1LoadoutTier','team2LoadoutTier'],inplace=True)
pStatsFinal.head()

,index,matchId,roundId,roundNumber,playerId,teamNumber,side,acs,kills,firstKills,deaths,firstDeaths,assists,damage,headshots,bodyshots,legshots,plants,defusals,clutches,clutchOpponents,clutchOpportunities,impact,kastRounds,playerTeam,map,playerStats,agentStats,firstDuelTime,winningTeamNumber,teamLoadout,enemyLoadout
0,0,58155,906584,1,835,2,atk,272,1,1,1,0,0,126,1,1,1,1,0,0,0,0,0.11063158287637609,1,Knights,Ascent,XXiF,Sage,28277.0,2,0.0,0.0
1,1,58155,906584,1,1961,1,def,734,3,0,1,0,0,270,2,2,0,0,0,0,3,1,0.06115419295030323,1,Cloud9,Ascent,xeta,Sova,NaN,2,0.0,0.0
2,2,58155,906584,1,2213,1,def,20,0,0,1,0,0,20,0,2,0,0,0,0,0,0,-0.08521074372468429,0,Cloud9,Ascent,mitch,Killjoy,NaN,2,0.0,0.0
3,3,58155,906584,1,2810,2,atk,183,0,0,0,0,2,183,1,1,0,0,0,0,0,0,0,1,Knights,Ascent,Genghsta,Sova,NaN,2,0.0,0.0
4,4,58155,906584,1,3340,2,atk,792,4,0,0,0,0,397,4,4,0,0,0,0,0,0,0.42696423585487474,1,Knights,Ascent,ban,Jett,NaN,2,0.0,0.0


# CREATE PLANTS / ULT KILL DF

In [24]:
events.head(30)
# len(events)

,index,roundId,roundNumber,roundTimeMillis,killId,tradedByKillId,tradedForKillId,bombId,resId,playerId,referencePlayerId,eventType,damageType,weaponId,ability,impact,attackingWinProbabilityBefore,attackingWinProbabilityAfter,attackingTeamNumber
0,0,906584,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,start,None,NaN,None,0,0.4884476800754307,0.4884476800754307,2
1,1,906584,1,21010,NaN,NaN,NaN,717472.0,NaN,835.0,NaN,plant,None,NaN,None,0,0.5992687369089558,0.5992687369089558,2
2,2,906584,1,28277,7632167.0,NaN,NaN,NaN,NaN,835.0,3599.0,kill,weapon,11.0,None,0.18277501265111362,0.6014724906183639,0.7842475032694776,2
3,3,906584,1,29960,7632163.0,NaN,NaN,NaN,NaN,3340.0,13591.0,kill,weapon,10.0,None,0.08225037622256559,0.7842475032694776,0.8664978794920432,2
4,4,906584,1,33947,7632164.0,NaN,NaN,NaN,NaN,3340.0,2213.0,kill,weapon,10.0,None,0.08521074372468429,0.8663920493405218,0.9516027930652061,2
5,5,906584,1,39845,7632168.0,NaN,NaN,NaN,NaN,1961.0,3894.0,kill,weapon,12.0,None,0.03620138165895892,0.9587237474530883,0.9225223657941294,2
6,6,906584,1,40678,7632169.0,NaN,NaN,NaN,NaN,1961.0,835.0,kill,weapon,12.0,None,0.07214342977473753,0.9259557233756542,0.8538122936009167,2
7,7,906584,1,44771,7632165.0,NaN,NaN,NaN,NaN,3340.0,4639.0,kill,weapon,10.0,None,0.09899708791569006,0.8538122936009167,0.9528093815166068,2
8,8,906584,1,45561,7632170.0,7632166.0,NaN,NaN,NaN,1961.0,3831.0,kill,weapon,12.0,None,0.11331540950854158,0.9528093815166068,0.8394939720080652,2
9,9,906584,1,46691,7632166.0,NaN,7632170.0,NaN,NaN,3340.0,1961.0,kill,weapon,10.0,None,0.1605060279919348,0.8394939720080652,1,2


In [25]:
events.head()
psdf = pStatsFinal[['roundId','playerId','playerTeam','map','playerStats','agentStats','side']]
plants = pd.DataFrame()
ults = pd.DataFrame()
plants = events.merge(psdf,how='left',on=['roundId'])
plantsFinal = plants[['roundId','playerTeam','side','map']].dropna().drop_duplicates().reset_index()
plantsFinal.drop(columns='index', inplace=True)
plantsFinal.head()

,roundId,playerTeam,side,map
0,906584,Knights,atk,Ascent
1,906584,Cloud9,def,Ascent
2,906585,Knights,atk,Ascent
3,906585,Cloud9,def,Ascent
4,906586,Knights,atk,Ascent


In [26]:
planted = events[events['eventType'] == 'plant']

planted['plants'] = 1
planted.head()

defused = events[events['eventType'] == 'defuse']
defused['defuses'] = 1
defused.head()

planted = planted[['roundId','plants']]
defused = defused[['roundId','defuses']]

planted = planted.merge(defused,how='outer',on='roundId')
plantsFinal = plantsFinal.merge(planted,how='left',on='roundId')
plantsFinal.fillna(0,inplace=True)
    


C:\Users\quiet\AppData\Local\Temp/ipykernel_10624/1992136701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  planted['plants'] = 1
C:\Users\quiet\AppData\Local\Temp/ipykernel_10624/1992136701.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defused['defuses'] = 1


In [27]:
plantsFinal.head(50)

,roundId,playerTeam,side,map,plants,defuses
0,906584,Knights,atk,Ascent,1.0,0.0
1,906584,Cloud9,def,Ascent,1.0,0.0
2,906585,Knights,atk,Ascent,1.0,0.0
3,906585,Cloud9,def,Ascent,1.0,0.0
4,906586,Knights,atk,Ascent,0.0,0.0
5,906586,Cloud9,def,Ascent,0.0,0.0
6,906587,Knights,atk,Ascent,1.0,0.0
7,906587,Cloud9,def,Ascent,1.0,0.0
8,906588,Knights,atk,Ascent,1.0,1.0
9,906588,Cloud9,def,Ascent,1.0,1.0


In [28]:
""" PLANTS BY MATCHID
midf = pStatsFinal[['matchId','roundId']]
plantsFinal = plantsFinal.merge(midf,how='left',on='roundId')
plantsFinal = plantsFinal.drop_duplicates()
plantsFinal = plantsFinal.groupby(['playerTeam','side','matchId'],as_index=False)[['plants','defuses']].sum()
plantsFinal = plantsFinal.drop_duplicates()
len(plantsFinal)
plantsFinal.head()
"""

" PLANTS BY MATCHID\nmidf = pStatsFinal[['matchId','roundId']]\nplantsFinal = plantsFinal.merge(midf,how='left',on='roundId')\nplantsFinal = plantsFinal.drop_duplicates()\nplantsFinal = plantsFinal.groupby(['playerTeam','side','matchId'],as_index=False)[['plants','defuses']].sum()\nplantsFinal = plantsFinal.drop_duplicates()\nlen(plantsFinal)\nplantsFinal.head()\n"

In [29]:
len(ults)

0

In [31]:
ults = events.merge(psdf,how='left',on=['roundId','playerId'])
ults = ults[ults['damageType'] == 'ability']
ults = ults[['roundId','playerStats','agentStats','ability','side','playerTeam','map']].reset_index().drop(columns='index')
ults.drop_duplicates(inplace=True)
len(ults)
#ults.head()

1018

# XVY

In [32]:
#append team name and map to xvy
xvy['teamName'] = xvy['teamId'].map(teamDict)
xvy = xvy.merge(matches, how='left', on='matchId')

In [33]:
xvy.drop(['index'],axis=1, inplace=True)
xvy.drop_duplicates(inplace=True)
xvy.head()

,matchId,teamId,teamNumber,side,situation,team1Count,team2Count,delta,wins,losses,teamName,map
0,58735,588,1,atk,5v1,5,1,4,1,0,Knights,Split
1,58736,588,1,atk,5v2,5,2,3,1,0,Knights,Breeze
2,58735,588,1,def,5v2,5,2,3,1,0,Knights,Split
3,58736,588,1,atk,4v1,4,1,3,1,0,Knights,Breeze
4,58735,588,1,def,5v3,5,3,2,1,0,Knights,Split


In [34]:
xvy.to_sql('xvyClean',conn, if_exists='replace')
plantsFinal.to_sql('plants',conn, if_exists='replace')
ults.to_sql('abilityKills',conn,if_exists='replace')
pStatsFinal.to_sql('pStatsClean',conn, if_exists='replace')

In [35]:
conn.close()